# Music name generator
Well here we are. I'm going to find THE best music name, thanks to machine learning

In [22]:
import numpy as np 
import pandas as pd 
import string
import matplotlib.pyplot as plt

import os
for dirname, _, filenames in os.walk('/home/goznalo/Programming/Python/musicnames'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


%matplotlib inline

/home/goznalo/Programming/Python/musicnames/artists.csv
/home/goznalo/Programming/Python/musicnames/miusic.ipynb
/home/goznalo/Programming/Python/musicnames/clearedlist2.csv
/home/goznalo/Programming/Python/musicnames/.git/HEAD
/home/goznalo/Programming/Python/musicnames/.git/config
/home/goznalo/Programming/Python/musicnames/.git/COMMIT_EDITMSG
/home/goznalo/Programming/Python/musicnames/.git/description
/home/goznalo/Programming/Python/musicnames/.git/index
/home/goznalo/Programming/Python/musicnames/.git/info/exclude
/home/goznalo/Programming/Python/musicnames/.git/hooks/pre-push.sample
/home/goznalo/Programming/Python/musicnames/.git/hooks/pre-applypatch.sample
/home/goznalo/Programming/Python/musicnames/.git/hooks/pre-rebase.sample
/home/goznalo/Programming/Python/musicnames/.git/hooks/commit-msg.sample
/home/goznalo/Programming/Python/musicnames/.git/hooks/fsmonitor-watchman.sample
/home/goznalo/Programming/Python/musicnames/.git/hooks/applypatch-msg.sample
/home/goznalo/Programm

### Importing the data
Let's import the data from the kaggle dataset music-artists-popularity, using the Pandas library.

In [23]:
filename = '/home/goznalo/Programming/Python/musicnames/artists.csv'
dataset = pd.read_csv(filename, usecols = [2]) # Obtain the artist name column.

/usr/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [24]:
dataset.head # preview of the data

<bound method NDFrame.head of                  artist_lastfm
0                     Coldplay
1                    Radiohead
2        Red Hot Chili Peppers
3                      Rihanna
4                       Eminem
...                        ...
1466078                    NaN
1466079                    NaN
1466080                    NaN
1466081                    NaN
1466082                    NaN

[1466083 rows x 1 columns]>

### Clearing, part 1
We don't want any of the NaN entries, nor duplicate names. We will also convert it to a numpy array for posterior transformations.

In [25]:
df = dataset.dropna(axis = 0,how = 'any',thresh = None).drop_duplicates(subset=None) #removing NaN's and converting to numpy.
names = np.squeeze(np.asarray(df))

In [26]:
numnames = names.shape[0]

In [27]:
names[0]

'Coldplay'

### Clearing, part 2
We will now remove all those names using non-standard characters. What I mean by non-standard is all those characters not being latin ones, nor punctuation ones, nor digits, nor spaces. For instance, we will get rid of those using chinese characters or greek letters.

We quickly run intro trouble, as the .isalnum() method applied to these non-latin characters returns True. We need to come up with a solution. One way around this is by using the string module. With it, the string.printable characters are those we want to allow in our names, therefore we make a set out of them, which will be used against the names of the database in a loop.

In [28]:
validchars = {}
validchars[0] = set(string.printable)
print(validchars[0])

{'E', 'S', 't', 'D', 'z', 'q', 'B', '1', '^', 'f', 'k', '5', 'a', 'Y', 'j', '"', '2', 'P', 'g', 'u', '|', '(', '*', 'r', '<', 'Z', '7', 'o', 'C', '~', ',', 'd', '0', 'h', ':', '}', 'J', '\x0b', 'Q', 'V', 'c', '.', '9', 'x', "'", '\x0c', '&', '#', '\n', 'F', 'X', 'W', 'b', '/', '_', 'U', '3', '8', '4', 'p', 'O', 'K', 'y', '@', 'H', 'l', 'w', 'L', ';', '%', 'i', ']', 'n', '\\', 'm', '[', ')', '?', '6', 'G', 'I', 'v', '>', ' ', '\t', '{', 'T', 's', 'A', 'N', '\r', '$', 'e', '-', 'M', '=', 'R', '+', '!', '`'}


Now comes the loop. An initial approach was creating a new object, a numpy array, to which we would append each valid name, therefore discarding the rest of the names with invalid characters. However, the appending operation makes the algorithm take exponential time of completion. Instead, we can save the index of each invalid word in a list, which we then feed to the np.delete() function to remove those entries of the "names" array

In [29]:
clearedlist = {}
def deletechars(validcharacters, listofnames, verbose=False):  # We define it as a function, as we will need it in the next steps
    i = 0 # counts each iteration
    j = 0 # counts each invalid word
    deletelist = []
    for name in listofnames:
        if not all(char in validcharacters for char in name):
            deletelist.append(i)
            if j%10000 == 0 and verbose:
                print("invalid: " + name)
            j += 1
        i += 1
        if i%50000 == 0 and verbose:
            print(str(i) + " cases inspected.")
    return np.delete(names, deletelist)

clearedlist[0] = deletechars(validchars[0], names) #by default, verbose = False (no output)

In [30]:
clearedlist[0][0:22] # Check that Beyoncé has been correctly removed.

array(['Coldplay', 'Radiohead', 'Red Hot Chili Peppers', 'Rihanna',
       'Eminem', 'The Killers', 'Kanye West', 'Nirvana', 'Muse', 'Queen',
       'Foo Fighters', 'Linkin Park', 'Lady Gaga', 'The Rolling Stones',
       'Daft Punk', 'Green Day', 'Katy Perry', 'The Beatles', 'Oasis',
       'Gorillaz', 'Michael Jackson', 'Maroon 5'], dtype=object)

In [31]:
np.savetxt("clearedlist1.csv", clearedlist[0], delimiter=",", fmt='%s') #save the list to a csv file.

### Clearing, part 3: choose your own adventure
We will now distinguish four different cases which can be studied, with decreasing complexity. 
1. The full list, as is.
2. The list, having removed punctuation characters and digits.
3. The list, having removed both punctuation characters, digits and names with more than two words.
4. The list, having removed both punctuation characters, digits and names with more than one word.

Case 2 is easy to implement, just applying the previously defined deletechars() function, specifying punctuation characters as invalid (allowing whitespaces). Case 4 is also straightforward to implement, as string.ascii_letters considers any whitespace invalid. Case 3 entails splitting each name using a "space" delimiter, and then discarding them lengthwise.

In [32]:
case = 3 #change to whichever of the above

In [33]:
validchars[1] = set(string.ascii_letters).union(" ")
validchars[2] = set(string.ascii_letters).union(" ") #we won't use it here, but we will count the characters with this.
validchars[3] = set(string.ascii_letters)
    
if (case == 2 or case == 3 or case == 4):
    clearedlist[1] = deletechars(validchars[1], clearedlist[0])
    np.savetxt("clearedlist2.csv", clearedlist[1], delimiter=",", fmt='%s') 
    
    if (case == 3 or case == 4): #as the function above, but indexing based on the length of string.split()
        i = 0 
        j = 0 
        deletelist[2] = []
        for name in clearedlist[1]:
            if len(name.split()) > 2:
                deletelist[2].append(i)
                if j%10000 == 0:
                    print("invalid: " + name)
                j += 1
            i += 1
            if i%50000 == 0:
                print(str(i) + " cases inspected (lengthwise).")
        clearedlist[2] = np.delete(clearedlist[1], deletelist[2])
        np.savetxt("clearedlist3.csv", clearedlist[2], delimiter=",", fmt='%s') 
        
        if case == 4: # string.ascii_letters considers whitespaces as invalid.
            clearedlist[3] = deletechars(validchars[3], clearedlist[2])
            np.savetxt("clearedlist4.csv", clearedlist[3], delimiter=",", fmt='%s')


print(clearedlist[2][0:22])

invalid: Red Hot Chili Peppers
50000 cases inspected (lengthwise).
invalid: Monkey To Millionaire
100000 cases inspected (lengthwise).
invalid: Mono Electric Orchestra
150000 cases inspected (lengthwise).
invalid: Easy & Center of the Universe
200000 cases inspected (lengthwise).
invalid: St. Gun Khin May
250000 cases inspected (lengthwise).
invalid: Thirty Seconds Until Armageddon
300000 cases inspected (lengthwise).
invalid: Derrick & Tee
350000 cases inspected (lengthwise).
invalid: The Insect Explosion
400000 cases inspected (lengthwise).
invalid: The Raisin Pickers
450000 cases inspected (lengthwise).
invalid: Niek en Danny
500000 cases inspected (lengthwise).
invalid: Sioux Red Indian Tribes
550000 cases inspected (lengthwise).
invalid: Sabre & Prezident Brown
600000 cases inspected (lengthwise).
invalid: Acto De Venganza
650000 cases inspected (lengthwise).
invalid: 方正 & 許不了
700000 cases inspected (lengthwise).
invalid: Tater Family Travelling Circus
750000 cases inspected (leng

### Some more pre-processing, I guess

Now that we have our desired array of names, we need to prepare them for being inputted to a neural network. There are several steps to be taken: first, we will make all names lowercase, so as to use a dimensionally smaller encoding. Then, making use of the char_to_index dictionary and the keras.utils.to_categorical() function, we will create our one-hot vectors


In [34]:
datasize = len(clearedlist[case-1])
for i in range(datasize):
    clearedlist[case-1][i] = clearedlist[case-1][i].lower()

In [35]:
chars = sorted(list(dict.fromkeys([char.lower() for char in validchars[case-1]])))
numchars = len(chars)
char_to_index = { ch:ix for ix, ch in enumerate(chars)} #assigns numbers to characters.